# Lab 2: Debug a Broken Vanilla GAN (find 12+ issues)

In [ ]:
import torch, torchvision, torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import transforms

device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
transform=transforms.Compose([transforms.ToTensor()])  # BUG normalize
loader=DataLoader(torchvision.datasets.MNIST('./data',True,download=True,transform=transform),batch_size=256,shuffle=True,num_workers=2,pin_memory=True)
z_dim=100; g_lr=2e-2; d_lr=2e-5
class D(nn.Module):
    def __init__(self):
        super().__init__(); self.net=nn.Sequential(nn.Conv2d(1,32,4,2,1),nn.LeakyReLU(0.2,True),nn.Conv2d(32,64,4,2,1),nn.BatchNorm2d(64),nn.LeakyReLU(0.2,True),nn.Conv2d(64,1,7,1,0),nn.Sigmoid())
    def forward(self,x):return self.net(x).view(x.size(0),1)
class G(nn.Module):
    def __init__(self):
        super().__init__(); self.net=nn.Sequential(nn.ConvTranspose2d(100,128,4,1,0),nn.BatchNorm2d(128),nn.ReLU(True),nn.ConvTranspose2d(128,64,4,2,1),nn.BatchNorm2d(64),nn.ReLU(True),nn.ConvTranspose2d(64,1,4,2,1))  # BUG missing Tanh
    def forward(self,z): z=z.view(z.size(0),64,1,1); return self.net(z)  # BUG reshape
Dnet=D().to(device); Gnet=G().to(device)
crit=nn.BCEWithLogitsLoss(); opt_d=torch.optim.Adam(Dnet.parameters(),lr=d_lr,betas=(0.9,0.999)); opt_g=torch.optim.Adam(Gnet.parameters(),lr=g_lr,betas=(0.9,0.999))

In [ ]:
from tqdm import tqdm
for real,_ in tqdm(loader):
    real=real.to(device); b=real.size(0)
    z=torch.randn(b,z_dim,device=device); fake=Gnet(z.view(b,z_dim,1,1))
    loss_d=crit(Dnet(real),torch.zeros(b,1,device=device))+crit(Dnet(fake),torch.ones(b,1,device=device))  # BUG labels
    loss_d.backward(); opt_g.step()  # BUG
    z=torch.randn(b,z_dim,device=device); fake=Gnet(z.view(b,z_dim,1,1))
    loss_g=crit(Dnet(fake),torch.zeros(b,1,device=device))  # BUG non-sat
    loss_g.backward()  # BUG missing step
print('Now fix all the issues.')